Image colorization project :
We train a CNN to take in greyscale images of ... and output their colorful and plausible colorization.

Imports required :

torch

skimage ? only used for rgb - lab parsing. Maybe we can do that ourselves to make some differentiation with the original project?

numpy

matplotlib

PIL ? only used to open images to rgb. We can use another library

In [9]:
import matplotlib.pyplot as plt 
import numpy as np
from PIL import Image
from skimage import color

import torch
import torch.nn as nn
import torch.utils.data as data
import torch.nn.functional as F

Load data, augment it, transform it to LAB compute stats on colors

In [10]:
#Data loading 

In [11]:
#LAB transform

In [27]:
#Color stats

Q = 10 # to find
p = torch.rand(10) # to find


In [28]:
#Define pixel weight vector (class rebalancing)

# Set the parameters (from paper, need empirical value).
lambda_uniform = 1/2 
sigma = 5 # gaussian kernel parameter

# Compute a smooth version of the empirical pixel color distribution.
p_smooth = p # how to do that ? gaussian kernel ? to do

# Compute the weight vector.
pixelsWeights = torch.reciprocal((1 - lambda_uniform) * p_smooth + lambda_uniform / Q)

# Normalise the weight vector according to p_smooth (E[W] = 1).
E_W = torch.sum(p_smooth * pixelsWeights)
scale_factor = 1 / E_W
pixelsWeights = scale_factor * pixelsWeights 

Define and instantiate Convolutional NN

In [13]:
#NN def

Define optimizer

In [14]:
#ADAM

Define Loss

In [20]:
# v(Z_hw) weight in paper (section 2)

def getPixelsWeights(Z):
    W = torch.argmax(Z, dim=2)
    for i in range(W.size(dim=0)):
        for j in range(W.size(dim=1)):
            W[i,j] = pixelsWeights(W[i,j])

    return W


In [21]:
#loss function 

def multinomialCrossEntropyLoss(Z_estimate, Z):
    W = getPixelsWeights(Z)
    L = - sum(W * torch.sum(Z * torch.log10(Z_estimate), dim=2))

    return L
    


Main training loop and additional function TBD